# CrewAI Flows Tutorial

> Aprende cómo crear y gestionar flujos de trabajo de IA usando CrewAI Flows.

## Descripción General

CrewAI Flows es una característica poderosa diseñada para simplificar la creación y gestión de flujos de trabajo de IA. Los Flows permiten a los desarrolladores combinar y coordinar tareas de programación y Crews de manera eficiente, proporcionando un marco robusto para construir automatizaciones de IA sofisticadas.

Los Flows te permiten crear flujos de trabajo estructurados y dirigidos por eventos. Proporcionan una forma fluida de conectar múltiples tareas, gestionar el estado y controlar el flujo de ejecución en tus aplicaciones de IA. Con Flows, puedes diseñar e implementar fácilmente procesos de múltiples pasos que aprovechan todo el potencial de las capacidades de CrewAI.

### Características Principales:

1. **Creación Simplificada de Flujos de Trabajo**: Encadena fácilmente múltiples Crews y tareas para crear flujos de trabajo de IA complejos.

2. **Gestión de Estado**: Los Flows hacen que sea súper fácil gestionar y compartir estado entre diferentes tareas en tu flujo de trabajo.

3. **Arquitectura Dirigida por Eventos**: Construido sobre un modelo dirigido por eventos, permitiendo flujos de trabajo dinámicos y responsivos.

4. **Control de Flujo Flexible**: Implementa lógica condicional, bucles y ramificación dentro de tus flujos de trabajo.

## Instalación y Configuración

Primero, necesitamos instalar las dependencias necesarias:

In [1]:
# Instalar dependencias
# !pip install crewai python-dotenv litellm onnxruntime

In [2]:
# Configurar variables de entorno
import os
from dotenv import load_dotenv

# Cargar variables de entorno desde archivo .env
load_dotenv()

# Verificar que la API key esté configurada
if not os.getenv('OPENAI_API_KEY'):
    print("⚠️  ADVERTENCIA: OPENAI_API_KEY no encontrada en variables de entorno")
    print("Por favor, crea un archivo .env con tu OPENAI_API_KEY")
else:
    print("✅ OPENAI_API_KEY configurada correctamente")

✅ OPENAI_API_KEY configurada correctamente


## Primeros Pasos

Vamos a crear un Flow simple donde usarás OpenAI para generar una ciudad aleatoria en una tarea y luego usar esa ciudad para generar un dato curioso en otra tarea.

In [3]:
from crewai.flow.flow import Flow, listen, start
from dotenv import load_dotenv
from litellm import completion


class ExampleFlow(Flow):
    model = "gpt-4o-mini"

    @start()
    def generate_city(self):
        print("Iniciando flow")
        # Cada estado del flow recibe automáticamente un ID único
        print(f"Flow State ID: {self.state['id']}")

        response = completion(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": "Devuelve el nombre de una ciudad aleatoria en el mundo.",
                },
            ],
        )

        random_city = response["choices"][0]["message"]["content"]
        # Almacenar la ciudad en nuestro estado
        self.state["city"] = random_city
        print(f"Ciudad Aleatoria: {random_city}")

        return random_city

    @listen(generate_city)
    def generate_fun_fact(self, random_city):
        response = completion(
            model=self.model,
            messages=[
                {
                    "role": "user",
                    "content": f"Dime un dato curioso sobre {random_city}",
                },
            ],
        )

        fun_fact = response["choices"][0]["message"]["content"]
        # Almacenar el dato curioso en nuestro estado
        self.state["fun_fact"] = fun_fact
        return fun_fact


# Crear y ejecutar el flow
flow = ExampleFlow()
flow.plot()
result = flow.kickoff()

print(f"\nDato curioso generado: {result}")

ValueError: The onnxruntime python package is not installed. Please install it with `pip install onnxruntime`

### Explicación del Ejemplo

En el ejemplo anterior, hemos creado un Flow simple que genera una ciudad aleatoria usando OpenAI y luego genera un dato curioso sobre esa ciudad. El Flow consta de dos tareas: `generate_city` y `generate_fun_fact`. La tarea `generate_city` es el punto de partida del Flow, y la tarea `generate_fun_fact` escucha la salida de la tarea `generate_city`.

Cada instancia de Flow recibe automáticamente un identificador único (UUID) en su estado, lo que ayuda a rastrear y gestionar las ejecuciones del flow. El estado también puede almacenar datos adicionales (como la ciudad generada y el dato curioso) que persisten durante toda la ejecución del flow.

Cuando ejecutas el Flow, hará lo siguiente:

1. Generar un ID único para el estado del flow
2. Generar una ciudad aleatoria y almacenarla en el estado
3. Generar un dato curioso sobre esa ciudad y almacenarlo en el estado
4. Imprimir los resultados en la consola

El ID único del estado y los datos almacenados pueden ser útiles para rastrear las ejecuciones del flow y mantener el contexto entre tareas.

**Nota:** Asegúrate de haber configurado tu archivo `.env` para almacenar tu `OPENAI_API_KEY`. Esta clave es necesaria para autenticar las solicitudes a la API de OpenAI.

## Decoradores de Flow

### @start()

El decorador `@start()` se usa para marcar un método como el punto de partida de un Flow. Cuando se inicia un Flow, todos los métodos decorados con `@start()` se ejecutan en paralelo. Puedes tener múltiples métodos de inicio en un Flow, y todos se ejecutarán cuando se inicie el Flow.

### @listen()

El decorador `@listen()` se usa para marcar un método como un listener para la salida de otra tarea en el Flow. El método decorado con `@listen()` se ejecutará cuando la tarea especificada emita una salida. El método puede acceder a la salida de la tarea que está escuchando como un argumento.

#### Uso

El decorador `@listen()` se puede usar de varias maneras:

1. **Escuchar un Método por Nombre**: Puedes pasar el nombre del método que quieres escuchar como una cadena. Cuando ese método se complete, se activará el método listener.

2. **Escuchar un Método Directamente**: Puedes pasar el método en sí. Cuando ese método se complete, se activará el método listener.

In [ ]:
# Ejemplo de uso de @listen con nombre de método
class ListenExampleFlow(Flow):
    
    @start()
    def generate_city(self):
        return "Madrid"
    
    @listen("generate_city")  # Usando nombre de método
    def generate_fun_fact(self, city):
        return f"Dato curioso sobre {city}"

# Ejemplo de uso de @listen con método directo
class ListenDirectExampleFlow(Flow):
    
    @start()
    def generate_city(self):
        return "Barcelona"
    
    @listen(generate_city)  # Usando método directo
    def generate_fun_fact(self, city):
        return f"Dato curioso sobre {city}"

print("Ejemplo con nombre de método:")
flow1 = ListenExampleFlow()
result1 = flow1.kickoff()
print(result1)

print("\nEjemplo con método directo:")
flow2 = ListenDirectExampleFlow()
result2 = flow2.kickoff()
print(result2)

## Salida del Flow

Acceder y manejar la salida de un Flow es esencial para integrar tus flujos de trabajo de IA en aplicaciones o sistemas más grandes. CrewAI Flows proporciona mecanismos directos para recuperar la salida final, acceder a resultados intermedios y gestionar el estado general de tu Flow.

### Recuperando la Salida Final

Cuando ejecutas un Flow, la salida final está determinada por el último método que se complete. El método `kickoff()` devuelve la salida de este último método.

In [ ]:
from crewai.flow.flow import Flow, listen, start

class OutputExampleFlow(Flow):
    @start()
    def first_method(self):
        return "Salida del primer método"

    @listen(first_method)
    def second_method(self, first_output):
        return f"Segundo método recibió: {first_output}"


flow = OutputExampleFlow()
flow.plot("my_flow_plot")
final_output = flow.kickoff()

print("---- Salida Final ----")
print(final_output)

### Accediendo y Actualizando el Estado

Además de recuperar la salida final, también puedes acceder y actualizar el estado dentro de tu Flow. El estado se puede usar para almacenar y compartir datos entre diferentes métodos en el Flow.

In [ ]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel

class ExampleState(BaseModel):
    counter: int = 0
    message: str = ""

class StateExampleFlow(Flow[ExampleState]):

    @start()
    def first_method(self):
        self.state.message = "Hola desde el primer método"
        self.state.counter += 1

    @listen(first_method)
    def second_method(self):
        self.state.message += " - actualizado por el segundo método"
        self.state.counter += 1
        return self.state.message

flow = StateExampleFlow()
flow.plot("my_flow_plot")
final_output = flow.kickoff()
print(f"Salida Final: {final_output}")
print("Estado Final:")
print(flow.state)

## Gestión del Estado del Flow

Gestionar el estado de manera efectiva es crucial para construir flujos de trabajo de IA confiables y mantenibles. CrewAI Flows proporciona mecanismos robustos tanto para la gestión de estado estructurado como no estructurado, permitiendo a los desarrolladores elegir el enfoque que mejor se adapte a las necesidades de su aplicación.

### Gestión de Estado No Estructurado

En la gestión de estado no estructurado, todo el estado se almacena en el atributo `state` de la clase `Flow`. Este enfoque ofrece flexibilidad, permitiendo a los desarrolladores agregar o modificar atributos de estado sobre la marcha sin definir un esquema estricto.

In [ ]:
from crewai.flow.flow import Flow, listen, start

class UnstructuredExampleFlow(Flow):

    @start()
    def first_method(self):
        # El estado incluye automáticamente un campo 'id'
        print(f"State ID: {self.state['id']}")
        self.state['counter'] = 0
        self.state['message'] = "Hola desde el flow no estructurado"

    @listen(first_method)
    def second_method(self):
        self.state['counter'] += 1
        self.state['message'] += " - actualizado"

    @listen(second_method)
    def third_method(self):
        self.state['counter'] += 1
        self.state['message'] += " - actualizado de nuevo"

        print(f"Estado después del tercer método: {self.state}")


flow = UnstructuredExampleFlow()
flow.plot("my_flow_plot")
flow.kickoff()

### Gestión de Estado Estructurado

La gestión de estado estructurado aprovecha esquemas predefinidos para asegurar consistencia y seguridad de tipos en todo el flujo de trabajo. Al usar modelos como `BaseModel` de Pydantic, los desarrolladores pueden definir la forma exacta del estado, permitiendo mejor validación y autocompletado en entornos de desarrollo.

In [ ]:
from crewai.flow.flow import Flow, listen, start
from pydantic import BaseModel


class ExampleState(BaseModel):
    # Nota: el campo 'id' se agrega automáticamente a todos los estados
    counter: int = 0
    message: str = ""


class StructuredExampleFlow(Flow[ExampleState]):

    @start()
    def first_method(self):
        # Acceder al ID generado automáticamente si es necesario
        print(f"State ID: {self.state.id}")
        self.state.message = "Hola desde el flow estructurado"

    @listen(first_method)
    def second_method(self):
        self.state.counter += 1
        self.state.message += " - actualizado"

    @listen(second_method)
    def third_method(self):
        self.state.counter += 1
        self.state.message += " - actualizado de nuevo"

        print(f"Estado después del tercer método: {self.state}")


flow = StructuredExampleFlow()
flow.kickoff()

## Control de Flujo

### Lógica Condicional: `or`

La función `or_` en Flows te permite escuchar múltiples métodos y activar el método listener cuando cualquiera de los métodos especificados emita una salida.

In [ ]:
from crewai.flow.flow import Flow, listen, or_, start

class OrExampleFlow(Flow):

    @start()
    def start_method(self):
        return "Hola desde el método de inicio"

    @listen(start_method)
    def second_method(self):
        return "Hola desde el segundo método"

    @listen(or_(start_method, second_method))
    def logger(self, result):
        print(f"Logger: {result}")



flow = OrExampleFlow()
flow.plot("my_flow_plot")
flow.kickoff()

### Lógica Condicional: `and`

La función `and_` en Flows te permite escuchar múltiples métodos y activar el método listener solo cuando todos los métodos especificados emitan una salida.

In [ ]:
from crewai.flow.flow import Flow, and_, listen, start

class AndExampleFlow(Flow):

    @start()
    def start_method(self):
        self.state["greeting"] = "Hola desde el método de inicio"

    @listen(start_method)
    def second_method(self):
        self.state["joke"] = "¿Qué comen las computadoras? Microchips."

    @listen(and_(start_method, second_method))
    def logger(self):
        print("---- Logger ----")
        print(self.state)

flow = AndExampleFlow()
flow.plot()
flow.kickoff()

### Router

El decorador `@router()` en Flows te permite definir lógica de enrutamiento condicional basada en la salida de un método. Puedes especificar diferentes rutas basadas en la salida del método, permitiéndote controlar el flujo de ejecución dinámicamente.

In [ ]:
import random
from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel

class ExampleState(BaseModel):
    success_flag: bool = False

class RouterFlow(Flow[ExampleState]):

    @start()
    def start_method(self):
        print("Iniciando el flow estructurado")
        random_boolean = random.choice([True, False])
        self.state.success_flag = random_boolean

    @router(start_method)
    def second_method(self):
        if self.state.success_flag:
            return "success"
        else:
            return "failed"

    @listen("success")
    def third_method(self):
        print("Tercer método ejecutándose")

    @listen("failed")
    def fourth_method(self):
        print("Cuarto método ejecutándose")


flow = RouterFlow()
flow.plot("my_flow_plot")
flow.kickoff()

## Agregando Agentes a Flows

Los agentes se pueden integrar perfectamente en tus flows, proporcionando una alternativa ligera a los Crews completos cuando necesitas ejecución de tareas más simple y enfocada. Aquí tienes un ejemplo de cómo usar un Agente dentro de un flow para realizar investigación de mercado:

In [ ]:
import asyncio
from typing import Any, Dict, List

from pydantic import BaseModel, Field

from crewai.agent import Agent
from crewai.flow.flow import Flow, listen, start


# Definir un formato de salida estructurado
class MarketAnalysis(BaseModel):
    key_trends: List[str] = Field(description="Lista de tendencias de mercado identificadas")
    market_size: str = Field(description="Tamaño estimado del mercado")
    competitors: List[str] = Field(description="Competidores principales en el espacio")


# Definir estado del flow
class MarketResearchState(BaseModel):
    product: str = ""
    analysis: MarketAnalysis | None = None


# Crear una clase de flow
class MarketResearchFlow(Flow[MarketResearchState]):
    @start()
    def initialize_research(self) -> Dict[str, Any]:
        print(f"Iniciando investigación de mercado para {self.state.product}")
        return {"product": self.state.product}

    @listen(initialize_research)
    async def analyze_market(self) -> Dict[str, Any]:
        # Crear un Agente para investigación de mercado
        analyst = Agent(
            role="Analista de Investigación de Mercado",
            goal=f"Analizar el mercado para {self.state.product}",
            backstory="Eres un analista de mercado experimentado con experiencia en "
            "identificar tendencias de mercado y oportunidades.",
            verbose=True,
        )

        # Definir la consulta de investigación
        query = f"""
        Investiga el mercado para {self.state.product}. Incluye:
        1. Tendencias clave del mercado
        2. Tamaño del mercado
        3. Competidores principales

        Formatea tu respuesta según la estructura especificada.
        """

        # Ejecutar el análisis con formato de salida estructurado
        result = await analyst.kickoff_async(query, response_format=MarketAnalysis)
        if result.pydantic:
            print("resultado", result.pydantic)
        else:
            print("resultado", result)

        # Devolver el análisis para actualizar el estado
        return {"analysis": result.pydantic}

    @listen(analyze_market)
    def present_results(self, analysis) -> None:
        print("\nResultados del Análisis de Mercado")
        print("=====================")

        if isinstance(analysis, dict):
            # Si obtuvimos un dict con clave 'analysis', extraer el objeto de análisis real
            market_analysis = analysis.get("analysis")
        else:
            market_analysis = analysis

        if market_analysis and isinstance(market_analysis, MarketAnalysis):
            print("\nTendencias Clave del Mercado:")
            for trend in market_analysis.key_trends:
                print(f"- {trend}")

            print(f"\nTamaño del Mercado: {market_analysis.market_size}")

            print("\nCompetidores Principales:")
            for competitor in market_analysis.competitors:
                print(f"- {competitor}")
        else:
            print("No hay datos de análisis estructurado disponibles.")
            print("Análisis crudo:", analysis)


# Ejemplo de uso
async def run_flow():
    flow = MarketResearchFlow()
    flow.plot("MarketResearchFlowPlot")
    result = await flow.kickoff_async(inputs={"product": "Chatbots impulsados por IA"})
    return result


# Ejecutar el flow
if __name__ == "__main__":
    asyncio.run(run_flow())

## Visualización de Flows

Visualizar tus flujos de trabajo de IA puede proporcionar información valiosa sobre la estructura y las rutas de ejecución de tus flows. CrewAI ofrece una herramienta de visualización poderosa que te permite generar gráficos interactivos de tus flows, facilitando la comprensión y optimización de tus flujos de trabajo de IA.

### ¿Qué son los Plots?

Los Plots en CrewAI son representaciones gráficas de tus flujos de trabajo de IA. Muestran las diversas tareas, sus conexiones y el flujo de datos entre ellas. Esta visualización ayuda a entender la secuencia de operaciones, identificar cuellos de botella y asegurar que la lógica del flujo de trabajo se alinee con tus expectativas.

### Cómo Generar un Plot

CrewAI proporciona dos métodos convenientes para generar plots de tus flows:

#### Opción 1: Usando el Método `plot()`

Si estás trabajando directamente con una instancia de flow, puedes generar un plot llamando al método `plot()` en tu objeto flow. Este método creará un archivo HTML que contiene el plot interactivo de tu flow.

```python
# Asumiendo que tienes una instancia de flow
flow.plot("my_flow_plot")
```

Esto generará un archivo llamado `my_flow_plot.html` en tu directorio actual. Puedes abrir este archivo en un navegador web para ver el plot interactivo.

#### Opción 2: Usando la Línea de Comandos

Si estás trabajando dentro de un proyecto CrewAI estructurado, puedes generar un plot usando la línea de comandos. Esto es particularmente útil para proyectos más grandes donde quieres visualizar toda la configuración del flow.

```bash
crewai flow plot
```

Este comando generará un archivo HTML con el plot de tu flow, similar al método `plot()`. El archivo se guardará en tu directorio del proyecto, y puedes abrirlo en un navegador web para explorar el flow.

### Entendiendo el Plot

El plot generado mostrará nodos que representan las tareas en tu flow, con bordes dirigidos que indican el flujo de ejecución. El plot es interactivo, permitiéndote hacer zoom dentro y fuera, y pasar el cursor sobre los nodos para ver detalles adicionales.

Al visualizar tus flows, puedes obtener una comprensión más clara de la estructura del flujo de trabajo, facilitando la depuración, optimización y comunicación de tus procesos de IA a otros.

In [ ]:
# Ejemplo de generación de plot
class PlotExampleFlow(Flow):
    
    @start()
    def start_task(self):
        return "Tarea inicial completada"
    
    @listen(start_task)
    def process_task(self, input_data):
        return f"Procesado: {input_data}"
    
    @listen(process_task)
    def final_task(self, processed_data):
        return f"Finalizado: {processed_data}"

# Crear el flow y generar el plot
flow = PlotExampleFlow()
flow.plot("ejemplo_flow_plot")
print("✅ Plot generado como 'ejemplo_flow_plot.html'")

## Ejecutando Flows

Hay dos formas de ejecutar un flow:

### Usando la API del Flow

Puedes ejecutar un flow programáticamente creando una instancia de tu clase flow y llamando al método `kickoff()`:

```python
flow = ExampleFlow()
result = flow.kickoff()
```

### Usando la CLI

A partir de la versión 0.103.0, puedes ejecutar flows usando el comando `crewai run`:

```shell
crewai run
```

Este comando detecta automáticamente si tu proyecto es un flow (basado en la configuración `type = "flow"` en tu pyproject.toml) y lo ejecuta en consecuencia. Esta es la forma recomendada de ejecutar flows desde la línea de comandos.

Para compatibilidad hacia atrás, también puedes usar:

```shell
crewai flow kickoff
```

Sin embargo, el comando `crewai run` es ahora el método preferido ya que funciona tanto para crews como para flows.

## Próximos Pasos

Si estás interesado en explorar ejemplos adicionales de flows, tenemos una variedad de recomendaciones en nuestro repositorio de ejemplos. Aquí hay cuatro ejemplos específicos de flows, cada uno mostrando casos de uso únicos para ayudarte a hacer coincidir tu tipo de problema actual con un ejemplo específico:

1. **Email Auto Responder Flow**: Este ejemplo demuestra un bucle infinito donde un trabajo en segundo plano se ejecuta continuamente para automatizar respuestas de email. Es un gran caso de uso para tareas que necesitan realizarse repetidamente sin intervención manual. [Ver Ejemplo](https://github.com/crewAIInc/crewAI-examples/tree/main/email_auto_responder_flow)

2. **Lead Score Flow**: Este flow muestra agregar retroalimentación humana en el bucle y manejar diferentes ramas condicionales usando el router. Es un excelente ejemplo de cómo incorporar toma de decisiones dinámica y supervisión humana en tus flujos de trabajo. [Ver Ejemplo](https://github.com/crewAIInc/crewAI-examples/tree/main/lead-score-flow)

3. **Write a Book Flow**: Este ejemplo sobresale en encadenar múltiples crews juntos, donde la salida de un crew es usada por otro. Específicamente, un crew esboza un libro completo, y otro crew genera capítulos basados en el esbozo. Eventualmente, todo se conecta para producir un libro completo. Este flow es perfecto para procesos complejos de múltiples pasos que requieren coordinación entre diferentes tareas. [Ver Ejemplo](https://github.com/crewAIInc/crewAI-examples/tree/main/write_a_book_with_flows)

4. **Meeting Assistant Flow**: Este flow demuestra cómo transmitir un evento para activar múltiples acciones de seguimiento. Por ejemplo, después de que se complete una reunión, el flow puede actualizar un tablero de Trello, enviar un mensaje de Slack y guardar los resultados. Es un gran ejemplo de manejar múltiples resultados de un solo evento, haciéndolo ideal para sistemas de gestión de tareas y notificaciones integrales. [Ver Ejemplo](https://github.com/crewAIInc/crewAI-examples/tree/main/meeting_assistant_flow)

Al explorar estos ejemplos, puedes obtener información sobre cómo aprovechar CrewAI Flows para varios casos de uso, desde automatizar tareas repetitivas hasta gestionar procesos complejos de múltiples pasos con toma de decisiones dinámica y retroalimentación humana.

También puedes ver nuestro video de YouTube sobre cómo usar flows en CrewAI:

[![CrewAI Flows Tutorial](https://img.youtube.com/vi/MTb5my6VOT8/0.jpg)](https://www.youtube.com/watch?v=MTb5my6VOT8)

## Resumen

En este tutorial hemos cubierto:

1. **Conceptos básicos de CrewAI Flows** - Entendiendo qué son los flows y sus características principales
2. **Decoradores principales** - `@start()` y `@listen()` para crear flujos de trabajo
3. **Gestión de estado** - Tanto estructurado como no estructurado usando Pydantic
4. **Control de flujo** - Usando `or_`, `and_` y `@router()` para lógica condicional
5. **Integración de agentes** - Cómo usar agentes dentro de flows
6. **Visualización** - Cómo generar plots para entender mejor tus flows
7. **Ejecución** - Diferentes formas de ejecutar flows

CrewAI Flows proporciona un marco poderoso y flexible para crear flujos de trabajo de IA sofisticados. Con las herramientas y conceptos que hemos cubierto, estás listo para comenzar a construir tus propios flows personalizados.

¡Feliz programación con CrewAI! 🚀